In [60]:
import numpy as np
import json, sys, os, glob
from shutil import copyfile
from PIL import Image


In [62]:
def npz2img(path_to_unprocessed = '../unprocessed/',path_to_data = '../static/data/',config_path = '../config.json'):
    config = json.load(open(config_path,'r'))[0]
    imgs = glob.glob(path_to_unprocessed+'imgs/*.png')
    for img in imgs:
        img_name = img.split('/')[-1]
        if os.path.exists(path_to_data+'imgs/'+img_name):
            pass
        else:
            copyfile(path_to_unprocessed+'imgs/'+img_name, path_to_data+'imgs/'+img_name)

        if os.path.exists(path_to_data+'segz/'+img_name):
            pass
        else:
            np_img = np.load(path_to_unprocessed+'segz/'+img_name+'.npz')['arr_0']
            np_img = np.array([np_img,np_img,np_img])
            np_img = np.moveaxis(np_img, 0, -1)
            h,w,_ = np_img.shape

            for label in range(8):
                orig_color = (label,label,label)
                replacement_color = tuple(config['classes'][label]['rgb'])
                np_img[(np_img == orig_color).all(axis = -1)] = replacement_color
            np_img = Image.fromarray(np.uint8(np_img))
            np_img.save(path_to_data+'segz/'+img_name,"PNG")
    

In [72]:
npz2img()

In [68]:
def img2npz(path_to_npz = '../npz/',path_to_data = '../static/data/',config_path = '../config.json'):
    config = json.load(open(config_path,'r'))[0]
    imgs = glob.glob(path_to_data+'segz/*.png')
    for img in imgs:
        img_name = img.split('/')[-1]
        if os.path.exists(path_to_npz+'segz/'+img_name+'.npz'):
            pass
        else:
            np_img = np.array(Image.open(path_to_data+'segz/'+img_name))
            h,w,_ = np_img.shape
            
            for label in range(8):
                orig_color = tuple(config['classes'][label]['rgb'])
                replacement_color = (label,label,label)
                np_img[(np_img == orig_color).all(axis = -1)] = replacement_color
                
            np_img = np_img[:,:,0]
            np.savez(path_to_npz+'segz/'+img_name+'.npz', arr_0=np_img)
    

In [69]:
img2npz()